Codigo do tcc

In [1]:
!pip3 install transformers
!pip3 install dl-translate
!pip3 install emoji
!pip3 install tensorflow
!pip3 install numpy
!pip3 install tensorflow_hub

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import dl_translate as dlt

2023-08-21 09:15:49.033661: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 09:15:49.426335: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 09:15:49.428117: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 09:15:52.036972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/lincoln81/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mt = dlt.TranslationModel()

In [38]:
def translater_frases(frase):
  text_hi = frase
  translater = mt.translate(text_hi, source=dlt.lang.PORTUGUESE, target=dlt.lang.ENGLISH)
  return translater

def classification_frasesR(frase):
  input = tokenizerR.encode(frase, return_tensors="tf")
  output = modelR.predict(input)[0]
  prediction = tf.nn.softmax(output, axis=1).numpy()
  sentiment = np.argmax(prediction) 

  return sentiment

def classification_frasesM(frase):
  print(frase)
  inputs = tokenizerM(frase, return_tensors="tf")
  outputs = modelM(inputs)
  logits = outputs.logits
  sentiment = tf.argmax(logits, axis=1).numpy()[0]

  return sentiment


## Modelo de classificação de Tweets, treinado com 23 milhões de exemplos em inglês, sobre covid.

In [4]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizerR = AutoTokenizer.from_pretrained("rabindralamsal/BERTsent")

modelR = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/BERTsent")


2023-08-21 09:16:03.118682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-21 09:16:03.118933: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/BERTsent.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions 

In [6]:
example_tweet = "The NEET exams show our Govt in a poor light: unresponsiveness to genuine concerns; admit cards not delivered to aspirants in time; failure to provide centres in towns they reside, thus requiring unnecessary & risky travels. What a disgrace to treat our #Covid warriors like this!"
    
input = tokenizerR.encode(example_tweet, return_tensors="tf")
output = modelR.predict(input)[0]
prediction = tf.nn.softmax(output, axis=1).numpy()
sentiment = np.argmax(prediction)
    
print(prediction)
print(sentiment)

1/1 [==============================] - 7s 7s/step
[[0.97267216 0.02368475 0.00364307]]
0


## XLM-RoBERTa é uma versão multilíngue do RoBERTa. Ele é pré-treinado em 2,5 TB de dados CommonCrawl filtrados contendo 100 idiomas.

In [5]:
tokenizerM = AutoTokenizer.from_pretrained("xlm-roberta-base")
modelM = TFAutoModelForSequenceClassification.from_pretrained("xlm-roberta-base")

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
texto = "what a great day, what a beautiful day"
inputs = tokenizerM(texto, return_tensors="tf")

outputs = modelM(inputs)
logits = outputs.logits

sentimentos = ["Negative", "Neutral", "Positive"]
indice_sentimento = tf.argmax(logits, axis=1).numpy()[0]
sentimento_predito = sentimentos[indice_sentimento]

print(f"Predicted Sentiment: {sentimento_predito}")


Predicted Sentiment: Negative


In [7]:
dataset = pd.read_csv("./TweetSentBR.csv")

In [8]:
dataset.drop(columns = ['id', 'id_twitter'],inplace=True)

In [9]:
dataset

,text,sentiment
0,Que coisa linda! O Programa #encontro estava m...,1
1,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,quem viu aquela lutadora modela barbuda tatuad...,0
4,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...
11560,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11561,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11562,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11563,"Que orgulho de ti, @sportrecife! #Encontro",1


In [10]:
positive = dataset[dataset['sentiment'] == '1'].head(10)

In [11]:
positive['textEnglish'] = positive['text'].apply(translater_frases)

In [12]:
positive['sentimentEnglish'] = positive['textEnglish'].apply(classification_frasesR)

1/1 [==============================] - 0s 58ms/step


In [13]:
positive['sentimentEnglish'] = positive['sentimentEnglish'].replace(2, '1').replace(0, '-1').replace(1, '0')

In [14]:
nomepositivo= 'positive.xlsx'
positive.to_excel(nomepositivo, index=False)

In [15]:
negative = dataset[dataset['sentiment'] == '-1'].head(10)

In [16]:
negative['textEnglish'] = negative['text'].apply(translater_frases)

In [17]:
negative['sentimentEnglish'] = negative['textEnglish'].apply(classification_frasesR)

1/1 [==============================] - 0s 111ms/step


In [18]:
negative['sentimentEnglish'] = negative['sentimentEnglish'].replace(2, '1').replace(0, '-1').replace(1, '0')

In [19]:

negative.to_excel('negative.xlsx', index=False)

In [12]:
dataset = pd.read_csv("./datasetcompolaridade.csv")

In [13]:
dataset.drop(columns = ['Unnamed: 0', 'tweets_translater_unicamp'],inplace=True)

In [15]:
positive = dataset[dataset['polaridade'] == 1].head(100)

In [17]:
negative = dataset[dataset['polaridade'] == -1].head(100)

In [19]:
neutral = dataset[dataset['polaridade'] == 0].head(100)

In [23]:
positive['polaridadeClassificationPortugues'] = positive['tweets_original'].apply(classification_frasesM)

In [25]:
positive['polaridadeClassificationIngles'] = positive['tweets_translater_face'].apply(classification_frasesM)

In [27]:
positive['polaridadeClassificationPortugues'] = positive['polaridadeClassificationPortugues'].replace(2, 1).replace(0, -1).replace(1, 0)

In [29]:
positive['polaridadeClassificationIngles'] = positive['polaridadeClassificationIngles'].replace(2, 1).replace(0, -1).replace(1, 0)

In [30]:
positive

,tweets_original,tweets_translater_face,polaridade,polaridadeClassificationPortugues,polaridadeClassificationIngles
0,Que coisa linda! O Programa #encontro estava m...,What a beautiful thing! The #touch program was...,1.0,-1,-1
1,"Por mais #Encontro com as Irmãs Galvão, adorei...","For more I met with the Galvan Sisters, I love...",1.0,-1,-1
2,Mr. CATRA lançando sua nova música PPK CHORA ...,Mr. CATRA releases his new song PPK CHORA on k...,1.0,-1,-1
5,"Cibele arrasou, humilhou!! #VideoShowAoVivo","The cave has broken, humiliated!! #VideoShowAo...",1.0,-1,-1
9,A Cearamirinense mais famosa do Brasil no #Ede...,The most famous Cearamirinense in Brazil at #E...,1.0,-1,-1
...,...,...,...,...,...
209,mal começou o programa e eu já tô chorando #Al...,It’s hard to start the program and I’m already...,1.0,-1,-1
210,Braulio sempre com suas poesias 😍 #Encontro,Braulio always with his poems,1.0,-1,-1
212,#ConversaComBial\r\nAnsiosa pra assistir a con...,#ConversaComBial Anxiosa to watch today's talk...,1.0,-1,-1
216,Rafael ainda nem nasceu e já escuta #anjo #40...,"I'm not sure I'm going to do it, I'm not sure ...",1.0,-1,-1


In [31]:
negative['polaridadeClassificationPortugues'] = negative['tweets_original'].apply(classification_frasesM)
negative['polaridadeClassificationIngles'] = negative['tweets_translater_face'].apply(classification_frasesM)

In [32]:
negative['polaridadeClassificationPortugues'] = negative['polaridadeClassificationPortugues'].replace(2, 1).replace(0, -1).replace(1, 0)
negative['polaridadeClassificationIngles'] = negative['polaridadeClassificationIngles'].replace(2, 1).replace(0, -1).replace(1, 0)

In [33]:
negative

,tweets_original,tweets_translater_face,polaridade,polaridadeClassificationPortugues,polaridadeClassificationIngles
4,Tô passada com esse cara.... quanta merda pode...,How much shit can come out of someone’s mouth ...,-1.0,-1,-1
8,Esse japa não entendi porra nenhuma de orquíde...,This japa I didn't understand any of orchids. ...,-1.0,-1,-1
19,Aí vc fica até 01h00 assistindo #MasterChefBR ...,Here you stay until 1h00 watching #MasterChefB...,-1.0,-1,-1
21,Imagina que insuportável ter de dar de comer p...,Imagine how unbearable it is to have to give f...,-1.0,-1,-1
25,o prato do Vitor B ficou ruim MEU DEUS TO PASS...,The Vitor B's dish was bad my God to pass bad ...,-1.0,-1,-1
...,...,...,...,...,...
298,ahhhh... uma semana pra ter #MasterChefBR de n...,Ahhhh... a week to have #MasterChefBR again... :(,-1.0,-1,-1
304,A Paola kkkkkkkkkk\r\n#MasterChefBR,The Paola kkkkkkkk #MasterChefBR,-1.0,-1,-1
305,Não tá dando pra assistir MasterChef com esse ...,You don’t want to see MasterChef with that tim...,-1.0,-1,-1
309,"O crush da vida saiu, estou bem triste #Master...","The crush of life has gone out, I'm very sad #...",-1.0,-1,-1


In [35]:
neutral['polaridadeClassificationPortugues'] = neutral['tweets_original'].apply(classification_frasesM)

In [40]:
neutral.dropna(inplace=True)

In [41]:
neutral['polaridadeClassificationIngles'] = neutral['tweets_translater_face'].apply(classification_frasesM)

Who saw that fighter model tattooed? #MasterChefBR
In Minas sacolé is chup chup! #videohowaovivo
My MasterChef dish of enlated sardine would be playing it on the dish, cook rice and still put potato palma next to #MasterChefBR
Fatima Bernardes on the accent hahaha professional coherence #VideoShowAoVivo
Hello, I would like Fiuk to participate in some picture on #MaisVoce.
Tchau Victor B. It’s obvious that VC will go out. #MasterChefBR
I would like to have a #Encounter hj
The guy of the System of a Down now plays violin?
It’s just going to get... #VideoShowAoVivo
#MasterChefBR I think hj doesn't have the blue team's curse!!
She was sad, but Silvio managed to marry Patricia rs #DomingoLegal
Date is a upgrade to Crush #MasterChefBR
This is the only way to get rid of the weight of the body.
People from the novel Love of the Angel #DomingoLegal
#MasterChefBr Escape : one likes catch me: I catch, always
It was the Beautiful Gil in the Tchan clip? #AltasHoras hahaha
If the chain is so bad pq 

In [42]:
neutral['polaridadeClassificationPortugues'] = neutral['polaridadeClassificationPortugues'].replace(2, 1).replace(0, -1).replace(1, 0)
neutral['polaridadeClassificationIngles'] = neutral['polaridadeClassificationIngles'].replace(2, 1).replace(0, -1).replace(1, 0)

In [43]:
neutral

,tweets_original,tweets_translater_face,polaridade,polaridadeClassificationPortugues,polaridadeClassificationIngles
3,quem viu aquela lutadora modela barbuda tatuad...,Who saw that fighter model tattooed? #MasterCh...,0.0,-1,-1
6,Em Minas sacolé é chup chup! #videoshowaovivo,In Minas sacolé is chup chup! #videohowaovivo,0.0,-1,-1
7,Meu prato MasterChef de sardinha enlatada seri...,My MasterChef dish of enlated sardine would be...,0.0,-1,-1
13,Fatima Bernardes sobre o sotaque hahaha coerên...,Fatima Bernardes on the accent hahaha professi...,0.0,-1,-1
23,Olá gostaria da participação do Fiuk em algum...,"Hello, I would like Fiuk to participate in som...",0.0,-1,-1
...,...,...,...,...,...
669,#MasterChefBR espero que na próxima semana a M...,#MasterChefBR I hope that next week Miriam wil...,0.0,-1,-1
670,#Encontro Cuidado ao ingerir broto de samambai...,Be careful when eating samambae outbreak becau...,0.0,-1,-1
671,Lembrei q hj tem #MasterChefBR e já me deu fome🙄,I remember q hj has #MasterChefBR and it’s alr...,0.0,-1,-1
676,Sardinha é muito boa em cima da pizza caseira ...,Sardinia is very good over the home pizza #Mas...,0.0,-1,-1
